## Import Necessary Libraries

In [1]:
## Import necessary libraries
import pandas as pd
import numpy as np
import random 
from urllib.parse import quote
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from scipy.fftpack import fft
from sklearn.decomposition import PCA

## Import libraries for the model
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm.notebook import trange
import statistics
from sklearn.metrics import f1_score, classification_report

## Set path for saving model training results 
import os
os.makedirs('./result', exist_ok=True)

## Set Cuda for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Set random seed
def set_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

# Set seed
seed_val = 77
set_seed(seed_val)

cuda


## Selecting Data Columns
* Tag names are loaded in sequential order.
* The process of selecting the required tag names from the tag name list.

In [2]:
# Function to display tag names
def show_column(URL):
    
    # Load tag name data
    df = pd.read_csv(URL)
    
    # Convert to list format
    df = df.values.reshape(-1)
    
    return df.tolist()

In [3]:
## Set parameters for displaying tag names
table = 'bearing'

NAME_URL = f'http://127.0.0.1:5654/db/tql/datahub/api/v1/get_tag_names.tql?table={table}'

## Generate tag name list
name = show_column(NAME_URL)

In [4]:
name

['s1-c1',
 's1-c2',
 's1-c3',
 's1-c4',
 's1-c5',
 's1-c6',
 's1-c7',
 's1-c8',
 's2-c1',
 's2-c2',
 's2-c3',
 's2-c4',
 's3-c1',
 's3-c2',
 's3-c3',
 's3-c4']

## Converting TAG Name Format
* After checking all the Tag Names from the Nasa bearing dataset in the previous step, extract only the columns to be used and convert them into parameter format.
* Use tag names related to the s1-c5

In [5]:
# Set the desired tag names
tags = ['s1-c5']

# Wrap each item in the list with single quotes and separate with commas
tags_ = ",".join(f"'{tag}'" for tag in tags)

# Check the selected tag names
print(tags_)

's1-c5'


## Loading Nasa Bearing Dataset
* Load the train, validation, and test datasets separately when loading the data.
* The example focuses on anomaly detection for the 3rd bearing -> using 's1-c5' as the Tag Name.
* Label all states except for the faulty condition as normal for the labeling process.

In [6]:
# Set the status of each bearing based on time ranges
B1 ={
    "early" : ["2003-10-22 12:06:24" , "2003-10-23 09:14:13"],
    "suspect" : ["2003-10-23 09:24:13" , "2003-11-08 12:11:44"],
    "normal" : ["2003-11-08 12:21:44" , "2003-11-19 21:06:07"],
    "suspect_1" : ["2003-11-19 21:16:07" , "2003-11-24 20:47:32"],
    "imminent_failure" : ["2003-11-24 20:57:32","2003-11-25 23:39:56"]
}
B2 = {
    "early" : ["2003-10-22 12:06:24" , "2003-11-01 21:41:44"],
    "normal" : ["2003-11-01 21:51:44" , "2003-11-24 01:01:24"],
    "suspect" : ["2003-11-24 01:11:24" , "2003-11-25 10:47:32"],
    "imminient_failure" : ["2003-11-25 10:57:32" , "2003-11-25 23:39:56"]
}

B3 = {
    "early" : ["2003-10-22 12:06:24" , "2003-11-01 21:41:44"],
    "normal" : ["2003-11-01 21:51:44" , "2003-11-22 09:16:56"],
    "suspect" : ["2003-11-22 09:26:56" , "2003-11-25 10:47:32"],
    "Inner_race_failure" : ["2003-11-25 10:57:32" , "2003-11-25 23:39:56"]
}

B4 = {
    "early" : ["2003-10-22 12:06:24" , "2003-10-29 21:39:46"],
    "normal" : ["2003-10-29 21:49:46" , "2003-11-15 05:08:46"],
    "suspect" : ["2003-11-15 05:18:46" , "2003-11-18 19:12:30"],
    "Rolling_element_failure" : ["2003-11-19 09:06:09" , "2003-11-22 17:36:56"],
    "Stage_two_failure" : ["2003-11-22 17:46:56" , "2003-11-25 23:39:56"]
}

In [7]:
# Data loading parameter settings

# Set the tag table name
table = 'bearing'
# Set the tag names
name = quote(tags_, safe=":/")
# Set the time format 
timeformat = quote('2006-01-02 15:04:05.000000')
# Set the data start time
start_time = quote('2003-10-22 12:06:24')
# Set the data end time
end_time = quote('2003-11-25 23:39:56')

In [8]:
# Data loading function
def data_load(table, name, start_time, end_time, timeformat):
    
    # Load data 
    df = pd.read_csv(f'http://127.0.0.1:5654/db/tql/datahub/api/v1/select-rawdata.tql?table={table}&name={name}&start={start_time}&end={end_time}&timeformat={timeformat}')
    
    # Convert to data grouped by the time
    df = df.pivot_table(index='TIME', columns='NAME', values='VALUE', aggfunc='first').reset_index()
    
    # Set TIME column
    df['TIME'] = pd.to_datetime(df['TIME'], format='%Y-%m-%d %H:%M:%S.%f')
    
    # Group by seconds and count the number of records
    df_counts = df.groupby(df['TIME'].dt.floor('S')).size().reset_index(name='count')

    # Filter groups with the same number of records
    # Select the most common count values
    most_common_count = df_counts['count'].mode()[0]

    # Filter by the most common count value
    filtered_df_counts = df_counts[df_counts['count'] == most_common_count]

    # Convert filtered time values to a list
    filtered_times = filtered_df_counts['TIME'].tolist()

    # Select only the filtered time values from the original DataFrame
    filtered_data = df[df['TIME'].dt.floor('S').isin(filtered_times)]

    # Group by TIME
    # Round to the nearest second
    filtered_data_ = filtered_data.copy()
    filtered_data_.loc[:, 'TIME'] = filtered_data_['TIME'].dt.floor('S')
    grouped = filtered_data_.groupby('TIME')['s1-c5'].apply(list).reset_index()

    # Split the list into individual columns
    s1_c5_df = pd.DataFrame(grouped['s1-c5'].tolist())

    # Merge with the 'TIME' column
    result_df = pd.concat([grouped[['TIME']], s1_c5_df], axis=1)
    
    # Set labels
    # Assign labels based on abnormal time ranges for each channel data
    result_df['label'] = np.where((result_df['TIME'] >= "2003-11-25 10:57:32") & (result_df['TIME'] <= "2003-11-25 23:39:56"), 1, 0)
    
    result_df = result_df.drop(['TIME'], axis=1)

    return result_df

In [9]:
# Load data
df = data_load(table, name, start_time, end_time, timeformat)
df

,0,1,2,3,4,5,6,7,8,9,...,20471,20472,20473,20474,20475,20476,20477,20478,20479,label
0,-0.105,-0.049,-0.005,-0.100,-0.151,0.046,-0.132,-0.164,-0.110,-0.100,...,-0.088,-0.107,-0.078,-0.093,-0.200,-0.159,-0.237,-0.027,-0.002,0
1,-0.083,-0.132,-0.081,-0.022,-0.129,-0.110,-0.149,-0.168,-0.225,-0.217,...,-0.129,-0.149,-0.046,0.007,-0.132,-0.073,0.056,-0.186,-0.049,0
2,-0.122,-0.244,-0.156,-0.076,0.046,-0.098,-0.142,0.083,-0.195,-0.088,...,-0.107,-0.061,-0.090,-0.049,-0.125,-0.056,-0.137,-0.247,-0.229,0
3,-0.210,-0.125,-0.090,-0.215,-0.225,-0.139,-0.042,-0.090,-0.142,-0.232,...,-0.046,-0.195,-0.085,-0.112,-0.049,-0.146,-0.154,-0.220,-0.090,0
4,-0.088,-0.088,-0.110,-0.269,0.024,-0.054,-0.156,-0.205,-0.056,-0.132,...,0.044,-0.122,-0.115,-0.056,-0.078,-0.002,-0.342,-0.173,-0.161,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2150,-0.212,-0.095,0.044,0.005,-0.364,-0.239,-0.098,-0.066,-0.081,-0.164,...,-0.574,0.127,0.767,0.173,0.217,-0.271,0.085,0.364,-0.166,1
2151,0.432,0.217,-0.627,-0.308,0.073,0.107,-0.359,0.056,-0.159,-0.181,...,-0.161,-0.046,-0.098,-0.186,-0.251,-0.032,-0.017,-0.073,-0.437,1
2152,-0.112,-0.510,0.037,0.063,-0.115,-0.066,0.117,0.405,-0.374,-0.339,...,-0.188,-0.002,-0.085,-0.149,-0.195,-0.227,-0.095,-0.115,-0.293,1
2153,-0.593,0.366,-0.393,-0.312,-0.156,0.288,0.122,-0.105,0.242,-0.166,...,0.510,-0.776,-0.483,0.288,0.359,0.120,-0.173,-0.400,0.505,1


In [10]:
# Split the data into train, validation, and test sets
train, test = train_test_split(df, test_size=0.2, shuffle=False)
valid, test = train_test_split(test, test_size=0.5, shuffle=False)

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

## Data Preprocessing

* 1 hanning window
* 2 FFT
* 3 MinMax Scaling
* 4 PCA

### 1. Applying Hanning Window

In [11]:
# Hanning window function setup 
def set_hanning_window(sample_rate, df):
    
    # Generate Hanning window
    hanning_window = np.hanning(sample_rate)

    # Apply Hanning window to each row
    df_windowed = df.multiply(hanning_window, axis=1)
    
    return df_windowed

In [12]:
# Sampling period -> Number of data points per second
window_length = len(df.columns[:-1])

# Applying Hanning Window
train_ = set_hanning_window(window_length, train.iloc[:,:-1])
valid_ = set_hanning_window(window_length, valid.iloc[:,:-1])
test_ = set_hanning_window(window_length, test.iloc[:,:-1])

### 2. Applying FFT (Fast Fourier Transform)

In [13]:
# FFT transformation function
def change_fft(sample_rate, df):
    # Total number of samples in the signal
    N = sample_rate
    
    fft_results = np.zeros((df.shape[0], N // 2 + 1), dtype=float)
    
    # Apply FFT to each row
    for i in range(df.shape[0]):
        
        # Calculate FFT for each row
        yf = fft(df.iloc[i].values)
        
        # Compute the absolute value of the FFT results and normalize (only the meaningful part)
        fft_results[i] = 2.0 / N * np.abs(yf[:N // 2 + 1])
    
    # Convert FFT results to a DataFrame
    fft_df = pd.DataFrame(fft_results)
    
    return fft_df

In [14]:
# Sampling period -> Number of data points per second
sampling_rate = len(df.columns[:-1])

# Applying FFT (Fast Fourier Transform)
train_ = change_fft(sampling_rate, train_)
valid_ = change_fft(sampling_rate, valid_)
test_ = change_fft(sampling_rate, test_)

### 3. Applying MinMaxScaler

In [15]:
# Scaler Setup
scaler = MinMaxScaler()

# Apply Scaler
train_ = scaler.fit_transform(train_.values)
valid_ = scaler.transform(valid_.values)
test_ = scaler.transform(test_.values)

# Set DataFrames
train_scaled = pd.DataFrame(train_)
valid_scaled = pd.DataFrame(valid_)
test_scaled = pd.DataFrame(test_)

### 4. Applying PCA (Principal Component Analysis)

In [16]:
## Applying PCA
# Select principal components explaining 95% of the variance
pca = PCA(n_components=0.95)

# Apply PCA
train_scaled_ = pca.fit_transform(train_scaled)
valid_scaled_ = pca.transform(valid_scaled)
test_scaled_ = pca.transform(test_scaled)

# Set DataFrames
train_scaled_ = pd.DataFrame(train_scaled_)
valid_scaled_ = pd.DataFrame(valid_scaled_)
test_scaled_ = pd.DataFrame(test_scaled_)

# Add labels
train_scaled_['label'] = train['label'].values
valid_scaled_['label'] = valid['label'].values
test_scaled_['label'] = test['label'].values

print(train_scaled_['label'].value_counts())
print(valid_scaled_['label'].value_counts())
print(test_scaled_['label'].value_counts())

label
0    1724
Name: count, dtype: int64
label
0    215
Name: count, dtype: int64
label
0    181
1     35
Name: count, dtype: int64


## Dataset & Loader Setup

### Window Dataset Configuration
* To train on time series data, you need to set the window size and the sliding step.

* Window size: Determines how many time points to group together.
* Step size: The time interval by which the window moves.

In [17]:
# Sliding Window Dataset setup
class SlidingWindowDataset(Dataset):
    def __init__(self, data, window_size, step_size):
        self.data = data
        self.window_size = window_size
        self.step_size = step_size
        self.windows, self.labels = self._create_windows()
    
    # Set up sliding windows
    def _create_windows(self):
        windows = []
        labels = []
        for i in range(0, len(self.data) - self.window_size + 1, self.step_size):
            window = self.data.iloc[i:i + self.window_size, :-1].values  # Exclude the last column
            label_array = self.data.iloc[i:i + self.window_size, -1].values  # The last column is the label
            
            # Set the label to 1 if there is any abnormal value in the label array
            if (label_array == 1).any():
                label = 1  
            else:
                label = 0
                
            windows.append(torch.Tensor(window))
            labels.append(torch.Tensor([label]))  # Convert label to Tensor
        return windows, labels
    
    def __len__(self):
        return len(self.windows)
    
    def __getitem__(self, idx):
        return self.windows[idx], self.labels[idx]

In [18]:
# Sliding window configuration
window_size = 3
step_size = 1 

# Set up datasets 
train_ = SlidingWindowDataset(train_scaled_, window_size, step_size)
valid_ = SlidingWindowDataset(valid_scaled_, window_size, step_size)
test_ = SlidingWindowDataset(test_scaled_, window_size, step_size)

# Set up data loaders
train_dataloader = DataLoader(train_, batch_size=32, shuffle=False)
valid_dataloader = DataLoader(valid_, batch_size=1, shuffle=False)
test_dataloader = DataLoader(test_, batch_size=1, shuffle=False)

In [19]:
# Verify DataLoader application and check the shape of the input data
print(list(train_dataloader)[0][0].shape)

torch.Size([32, 3, 1487])


## Model Configuration
* Using LSTM AE model.

In [20]:
# LSTM Autoencoder class definition
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(LSTMAutoencoder, self).__init__()
        
        # Encoder LSTM
        self.encoder_lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.encoder_fc = nn.Linear(hidden_dim, 2*hidden_dim)
        
        # Decoder LSTM
        self.decoder_fc = nn.Linear(2*hidden_dim, hidden_dim)
        self.decoder_lstm = nn.LSTM(hidden_dim, input_dim, num_layers, batch_first=True)

    def forward(self, x):
        # Encoder part
        _, (h, _) = self.encoder_lstm(x)
        latent = self.encoder_fc(h[-1])
        
        # Decoder part
        hidden = self.decoder_fc(latent).unsqueeze(0).repeat(x.size(1), 1, 1).permute(1, 0, 2)
        output, _ = self.decoder_lstm(hidden)
        
        return output

In [21]:
# Model configuration parameters

# number of input data columns
# last number in print(list(train_dataloader)[0][0].shape)
input_dim = 1487

# LSMT hidden state size
hidden_dim = 256

# layer size
num_layers = 3

# Learning rate 
learning_rate = 0.01

# Model configuration
model = LSTMAutoencoder(input_dim, hidden_dim, num_layers).to(device)

# Configure loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Check the model architecture
print(model)

LSTMAutoencoder(
  (encoder_lstm): LSTM(1487, 256, num_layers=3, batch_first=True)
  (encoder_fc): Linear(in_features=256, out_features=512, bias=True)
  (decoder_fc): Linear(in_features=512, out_features=256, bias=True)
  (decoder_lstm): LSTM(256, 1487, num_layers=3, batch_first=True)
)


## Model Training

* Save the model with the Best Loss based on the training data during training.

In [22]:
# Initialize loss
train_loss = []
# Initialize total step
total_step = len(train_dataloader)
# Set number of epochs
epoch_in = trange(10, desc='training')
# Initialize best Loss value
best_Loss= np.inf

# Start model training
for epoch in epoch_in:
    model.to(device)
    model.train()
    running_loss = 0.0

    preds_ = []
    targets_ = []

    for batch_idx, train_data in enumerate(train_dataloader):

        inputs = train_data[0].to(device).float()

        optimizer.zero_grad()
        
        # Input to the model
        outputs = model(inputs)
 
        # Calculate loss 
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    train_loss.append(running_loss/total_step)
    print(f'\ntrain loss: {np.mean(train_loss)}')

    # Save the best model
    if best_Loss > np.mean(train_loss):
        best_Loss = np.mean(train_loss)
        torch.save(model, f'./result/Nasa_Bearing_LSTM_AE_General.pt')
        print('Model saved')
    epoch_in.set_postfix_str(f"epoch = {epoch}, best_Loss = {best_Loss}")

training:   0%|          | 0/10 [00:00<?, ?it/s]


train loss: 0.1874480980137984
Model saved

train loss: 0.18224455574872317
Model saved

train loss: 0.18051004501772516
Model saved

train loss: 0.17964278979019987
Model saved

train loss: 0.17912243715039006
Model saved

train loss: 0.17877553511456945
Model saved

train loss: 0.17852774798554719
Model saved

train loss: 0.17834190784574105
Model saved

train loss: 0.17819736530015498
Model saved

train loss: 0.1780817310705229
Model saved


## Threshold Setting
* Calculate the threshold using validation data
  * Max + K × Standard Deviation

In [23]:
# Load the best model
model_ = torch.load(f'./result/Nasa_Bearing_LSTM_AE_General.pt') 

In [24]:
# Calculate validation data reconstruction loss
valid_loss = []
with torch.no_grad():
    
    for batch_idx, valid_data in enumerate(valid_dataloader):

        inputs_val = valid_data[0].to(device).float()

        outputs_val = model_(inputs_val)
        loss = criterion(outputs_val, inputs_val)
        
        valid_loss.append(loss.item())
        
# Threshold setting
# The threshold should be adjusted according to your own criteria
# This threshold is composed of Max + K × Standard Deviation, and the K value is adjusted based on validation
threshold =  max(valid_loss) + 10*statistics.stdev(valid_loss)

print(threshold)

0.18937314544431605


## Model Testing

* Proceed with model testing on the test data based on the threshold calculated in the previous step.

In [25]:
# Apply the model to the test data
test_loss = []
test_label = []
with torch.no_grad():
    model_.eval()
    for batch_idx, test_data in enumerate(test_dataloader):

        inputs_test = test_data[0].to(device).float()
        label = test_data[1].to(device).long()

        outputs_test = model_(inputs_test)
        loss = criterion(outputs_test, inputs_test)
        
        test_loss.append(loss.item())
        test_label.append(label.item())

# Create a DataFrame for the test results
result = pd.DataFrame(test_loss, columns=['Reconst_Loss'])

# Set the actual labels
result['label'] = test_label

# Classify normal and abnormal based on each threshold
result['pred'] = np.where(result['Reconst_Loss'] > threshold, 1, 0)

## Model Performance Evaluation

In [26]:
# Print F1 Score based on testing data
print(classification_report(result['label'], result['pred']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       179
           1       1.00      1.00      1.00        35

    accuracy                           1.00       214
   macro avg       1.00      1.00      1.00       214
weighted avg       1.00      1.00      1.00       214



## Overfitting Check

* Calculate the F1 score using not only the test data but also the train and validation datasets.
* If the results for the train and validation are not satisfactory, it can be determined that the model is overfitting.

In [27]:
# Apply model to train data
train_loss = []
train_label = []
train_dataloader = DataLoader(train_, batch_size=1, shuffle=False)

with torch.no_grad():
    model_.eval()
    for batch_idx, train_data in enumerate(train_dataloader):

        inputs_train = train_data[0].to(device).float()
        label = train_data[1].to(device).long()

        outputs_train = model_(inputs_train)
        loss = criterion(outputs_train, inputs_train)
        
        train_loss.append(loss.item())
        train_label.append(label.item())

# Create a DataFrame for the train results
result = pd.DataFrame(train_loss, columns=['Reconst_Loss'])

# Set actual labels
result['label'] = train_label

# Classify normal and abnormal based on the threshold
result['pred'] = np.where(result['Reconst_Loss'] > threshold, 1, 0)

In [28]:
# Print F1 Score based on training data
print(classification_report(result['label'], result['pred'],labels=[0]))

              precision    recall  f1-score   support

           0       1.00      0.83      0.90      1722

   micro avg       1.00      0.83      0.90      1722
   macro avg       1.00      0.83      0.90      1722
weighted avg       1.00      0.83      0.90      1722



In [29]:
# Apply model on validation data
val_loss = []
val_label = []
with torch.no_grad():
    model_.eval()
    for batch_idx, valid_data in enumerate(valid_dataloader):

        inputs_val = valid_data[0].to(device).float()
        label = valid_data[1].to(device).long()

        outputs_val = model_(inputs_val)
        loss = criterion(outputs_val, inputs_val)
        
        val_loss.append(loss.item())
        val_label.append(label.item())

# Create a DataFrame for the validation results
result = pd.DataFrame(val_loss, columns=['Reconst_Loss'])

# Set the actual label
result['label'] = val_label

# Classify normal and abnormal based on the threshold
result['pred'] = np.where(result['Reconst_Loss'] > threshold, 1, 0)

In [30]:
# Print F1 Score based on validation data
print(classification_report(result['label'], result['pred'],labels=[0]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       213

    accuracy                           1.00       213
   macro avg       1.00      1.00      1.00       213
weighted avg       1.00      1.00      1.00       213

